In [1]:
import os
import tqdm
from PIL import Image

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import lightgbm as lgb

import torch
import torch.nn as nn
import torch, torch.nn as nn, torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, AdamW

import torchvision
from torchvision import models
from torchvision import datasets
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import Compose
from torchvision.transforms import transforms
from torchvision.transforms import RandomCrop, RandomHorizontalFlip, Normalize

import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
from tools.csv_preprocessed_util import preprocess_pipeline

writer = SummaryWriter()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. 데이터 불러오기

## 1) CSV 데이터 최종 전처리

In [2]:
df = pd.read_csv("../../csv/data_regression_clean.csv")
df.info() # 1391개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1391 entries, 0 to 1390
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1391 non-null   int64  
 1   id            1391 non-null   object 
 2   title         1391 non-null   object 
 3   detail        1391 non-null   object 
 4   condition     1391 non-null   object 
 5   is_completed  1391 non-null   bool   
 6   price         1391 non-null   float64
 7   location      1091 non-null   object 
 8   source        1391 non-null   object 
 9   model         1391 non-null   object 
 10  model_type    640 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 110.2+ KB


In [3]:
config = {
    # 사용할 칼럼만
    "select_cols": ["id", "condition", "is_completed", "location", "model", "model_type", "price"],

    # 범주형 라벨 인코딩 (id는 제외)
    "label_encode": {"cols": ["condition", "is_completed", "location", "model", "model_type"]},

    # price는 타깃이므로 따로 스케일링/인코딩 안함
    "final_cols": ["id", "condition", "is_completed", "location", "model", "model_type", "price"]
}

# fit 모드 (train 데이터)
df_processed, artifacts = preprocess_pipeline(df, mode="fit", config=config)

# val/test에 적용할 때
# df_val_processed, _ = preprocess_pipeline(df_val, mode="transform", artifacts=artifacts, config=config)

In [4]:
artifacts['label_encoders']['condition'].classes_

array(['사용감 많음', '사용감 적음', '새 상품'], dtype=object)

## 2) 이미지 데이터 불러오기

In [5]:
def load_images_by_uuid(image_root_path, uuid_list):
    """
    주어진 UUID 리스트에 해당하는 이미지들을 불러와 딕셔너리로 반환하는 함수입니다.
    Args:
        image_root_path (str): 이미지가 저장된 루트 디렉토리 경로 (예: 'C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\preprocessed\images')
        uuid_list (list): 이미지를 불러올 UUID 문자열 리스트.
    Returns:
        dict: {uuid: [Image1, Image2, ...]} 형식의 딕셔너리
    """
    image_data_dict = {}
    error_image_list = []
    count= 0
    for uuid in uuid_list:
        uuid_path = os.path.join(image_root_path, uuid)
        if not os.path.isdir(uuid_path):    
            error_image_list.append(uuid)
            print(f"경고: {uuid} 경로를 찾을 수 없습니다.")
            continue
        
        image_list = []
        for filename in os.listdir(uuid_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(uuid_path, filename)
                try:
                    img = Image.open(image_path).convert('RGB')
                    image_list.append(img)
                except Exception as e:
                    print(f"오류: {image_path} 이미지 파일을 불러올 수 없습니다. 오류: {e}")
        
        if image_list:
            image_data_dict[uuid] = image_list
        else:
            print(f"정보: {uuid_path} 폴더에 이미지가 없습니다.")
    
    return image_data_dict, error_image_list

target_uuids_list = df['id'].tolist()

image_base_path = '../../data/total_images'
uuid_images, error_image_list = load_images_by_uuid(image_base_path, target_uuids_list)
print(f"\n이미지들을 불러오는 중...")

uuid_images[target_uuids_list[0]]


이미지들을 불러오는 중...


[<PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>,
 <PIL.Image.Image image mode=RGB size=480x480>]

In [6]:
len(target_uuids_list)

1391

In [7]:
len(error_image_list)

0

In [8]:
# 이미지를 변환및 크롭 함수 정의
def get_image_transforms(target_size=224):
    """
    이미지 변환 파이프라인 정의
    여러 이미지 크롭 및 리사이즈 방법을 정의
    Compose를 사용하여 파이프라인을 만듭니다.
    """
    def custom_crop_and_resize(img):
        """
        이미지의 비율에 따라 크롭 또는 패딩을 적용하여 1:1 비율로 만드는 내부 함수입니다.
        """
        width, height = img.size
        # 가로 세로 비율 차이가 크면 크롭을 적용합니다.
        if abs(width / height - 1.0) > 0.1:
            if width > height:
                img = transforms.CenterCrop((height, height))(img)
            else:
                img = transforms.CenterCrop((width, width))(img)
        # 비율 차이가 작으면 패딩을 적용하여 정사각형으로 만듭니다.
        else:
            max_side = max(width, height)
            new_img = Image.new('RGB', (max_side, max_side), (0, 0, 0))
            new_img.paste(img, ((max_side - width) // 2, (max_side - height) // 2))
            img = new_img
        
        return img.resize((target_size, target_size))

    return Compose([
        transforms.Lambda(lambda img: custom_crop_and_resize(img)),
        transforms.ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# 2. 데이터 셋 만들기

In [9]:
# 데이터셋 및 데이터로더 준비
class CombinedDataset(torch.utils.data.Dataset):
    """
    이미지 데이터와 csv 데이터를 함께 처리하기 위한 필수적인 Dataset 클래스
    """
    def __init__(self, df, image_ids, target_id, train_ids, image_transform=None):
        self.df = df
        self.transform = image_transform
        self.image_ids = image_ids
        self.labels = df[target_id]

        self.tabular_data = self.df[train_ids]
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        uuid_path = os.path.join(image_base_path, image_id)
        filename = os.listdir(uuid_path)[0]
        image_path = os.path.join(uuid_path, filename)
        image = Image.open(image_path).convert('RGB')
        
        # 이미지 크롭 추가

        image_tensor = self.transform(image)
        
        tabular_tensor = torch.tensor(self.tabular_data.iloc[idx].values, dtype=torch.float32)
        tabular_tensor = tabular_tensor
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        return image_tensor, tabular_tensor, label

def prepare_data_and_loaders(df, target_id, train_ids, batch_size=32):
    """
    CSV 파일을 불러오고, 학습/검증 데이터셋으로 나눈 후, 데이터로더를 반환하는 함수입니다.
    """
    targets = df[target_id]
    df = df.dropna(subset=[target_id])
    
    # 'id' 컬럼을 리스트로 불러옵니다.
    id_list = df['id'].tolist()
    
    # 학습/검증 데이터로 분리
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)

    image_transforms = get_image_transforms()
    
    train_dataset = CombinedDataset(df=train_df, image_ids = train_df['id'].tolist() , target_id = target_id, train_ids = train_ids, image_transform=image_transforms)
    val_dataset = CombinedDataset(df=val_df, image_ids = val_df['id'].tolist(), target_id= target_id,train_ids = train_ids, image_transform=image_transforms)
    
    train_dataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataLoader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    tabular_data_size = len(train_dataset.tabular_data.columns)
    
    print(f"학습 데이터 크기: {len(train_df)}")
    print(f"검증 데이터 크기: {len(val_df)}")
    print(f"배치 사이즈: {batch_size}")
    print(f"전체 ID 개수: {len(id_list)}")
    
    return train_dataLoader, val_dataLoader, tabular_data_size, id_list

In [10]:
# 메인 실행 부분 (데이터 로더 호출)
train_loader, val_loader, tabular_size, id_list = prepare_data_and_loaders(df_processed, target_id = 'price', train_ids = ['is_completed', 'location', 'model', 'model_type', 'condition'], batch_size=32)

학습 데이터 크기: 1112
검증 데이터 크기: 279
배치 사이즈: 32
전체 ID 개수: 1391


In [11]:
image_tensor, tabular_tensor, label = next(iter(train_loader))
image_tensor[0]
image_tensor.shape

torch.Size([32, 3, 224, 224])

# 3. 모델 정의

In [12]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

# 2) 회귀용 헤드로 교체
in_f = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_f, 1)

In [13]:
# 3-1. 컨볼루션 파트 & 3-2. 회귀 파트
class CombinedModel(nn.Module):
    """
    이미지 특징과 csv 데이터를 결합하여 가격을 예측하는 회귀 모델입니다.
    """
    def __init__(self, tabular_data_size, tab_scale=1.0, img_scale=1.0, img_dim=64, tab_dim=256):
        super(CombinedModel, self).__init__()
        self.tab_scale = tab_scale
        self.img_scale = img_scale
        
        # 3-1. 컨볼루션 파트 (이미지 벡터화)
        self.conv_part = model
        
        # 컨볼루션 파트의 출력 벡터 크기를 계산합니다.
        with torch.no_grad():
            dummy_image = torch.randn(1, 3, 224, 224)
            conv_out_size = self.conv_part(dummy_image).numel()
            
        self.img_head = nn.Sequential(
            nn.Linear(conv_out_size, img_dim), nn.ReLU()
        )
        self.tab_head = nn.Sequential(
            nn.Linear(tabular_data_size, tab_dim), nn.ReLU()
        )            
        
        # 이미지 벡터와 csv 데이터 벡터를 합친 전체 특징 크기를 계산합니다.
        combined_features_size = img_dim + tab_dim
        
        # FC 파트
        self.reg_part = nn.Sequential(
            nn.Linear(combined_features_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        
    def forward(self, images, tabular_data):
        # 스케일은 그대로 두되, 투영 헤드를 통과시켜 차원을 바꾼 뒤 concat
        image_features = self.conv_part(images) * self.img_scale
        tab_features   = tabular_data * self.tab_scale

        image_features = self.img_head(image_features)  # (B, img_dim)
        tab_features   = self.tab_head(tab_features)    # (B, tab_dim)

        combined_features = torch.cat((image_features, tab_features), dim=1)
        output = self.reg_part(combined_features)
        
        print(f"image_features : {image_features.size()}")
        print(f"tab_features : {tab_features.size()}")
        print(f"combined_features : {combined_features.size()}")
        print(f"output : {output.size()}")
        
        return output

# 4. 모델 학습

In [14]:
# 모델 인스턴스화
if tabular_size is not None:
    model = CombinedModel(
        tabular_data_size=tabular_size,
        tab_scale=5.0,            
        img_scale=0.2,            
        img_dim=32,               
        tab_dim=256).to(device)
    
    print("\n모델 구조:")
    print(model)
    print(f"\n모델을 '{device}' 장치로 이동했습니다.")
else:
    print("\n데이터 로딩에 문제가 발생하여 모델을 인스턴스화할 수 없습니다.")


모델 구조:
CombinedModel(
  (conv_part): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
          

In [ ]:
# 모델 학습을 위한 하이퍼파라미터를 설정합니다.
num_epochs = 10
step = 0

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.SmoothL1Loss()  # or nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

writer = SummaryWriter()

# 학습 루프를 시작합니다.
print("모델 학습 시작...")
for epoch in range(num_epochs):
    
    # 모델을 학습 모드로 설정합니다.
    model.train()
    running_loss = 0.0
    
    for i, (images, tabular_data, labels) in enumerate(tqdm.tqdm(train_loader)):
        # 데이터를 지정된 장치(GPU 또는 CPU)로 이동합니다.
        images = images.to(device)
        tabular_data = tabular_data.to(device)
        labels = labels.to(device).float()

        # 옵티마이저의 기울기를 0으로 초기화합니다.
        optimizer.zero_grad()
        
        # 순전파(forward pass)를 수행하여 예측값을 얻습니다.
        outputs = model(images, tabular_data).squeeze(1)
        
        # 손실을 계산합니다.
        loss = criterion(outputs, labels.view(-1))
        writer.add_scalar('Loss/train', loss.item(), step)
        
        # 역전파(backward pass)를 수행하여 기울기를 계산합니다.
        loss.backward()
        
        # 옵티마이저를 통해 모델의 파라미터를 업데이트합니다.
        optimizer.step()
        step += 1
        
        running_loss += loss.item()
        
    print(f"에포크 [{epoch+1}/{num_epochs}], 손실(Loss): {running_loss / len(train_loader):.4f}")

print("모델 학습 완료!")


모델 학습 시작...


  0%|          | 0/35 [00:00<?, ?it/s]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


  3%|▎         | 1/35 [00:03<02:00,  3.56s/it]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


...

 60%|██████    | 21/35 [00:48<00:23,  1.67s/it]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


 63%|██████▎   | 22/35 [00:49<00:20,  1.56s/it]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


 66%|██████▌   | 23/35 [00:50<00:17,  1.50s/it]

# 5. 모델 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 모델 평가를 시작합니다.
print("모델 평가 시작...")

# 평가 단계에서는 기울기를 계산할 필요가 없으므로 torch.no_grad()를 사용합니다.
# 이는 메모리 사용량을 줄이고 연산 속도를 높여줍니다.
with torch.no_grad():
    # 모델을 평가 모드로 설정합니다.
    model.eval()    
    total_loss = 0.0
    y_true_all = []
    y_pred_all = []
    
    for images, tabular_data, labels in tqdm.tqdm(val_loader):

        # 데이터를 지정된 장치(GPU 또는 CPU)로 이동합니다.
        images = images.to(device)
        tabular_data = tabular_data.to(device)
        labels = labels.to(device).view(-1) # MSELoss를 위해 라벨의 형태를 [배치사이즈, 1]로 변경
        
        # 순전파를 수행하여 예측값을 얻습니다.
        outputs = model(images, tabular_data).squeeze(1)
        
        # 손실을 계산합니다.
        loss = criterion(outputs, labels)
        preds = outputs
        target = labels
        
        total_loss += loss.item()
        y_true_all.append(target.detach().cpu().numpy())
        y_pred_all.append(preds.detach().cpu().numpy())
        
    y_true_all = np.concatenate(y_true_all)  # (M,)
    y_pred_all = np.concatenate(y_pred_all)  # (M,)
        
    # 검증 데이터셋에 대한 평균 손실을 계산합니다.
    avg_loss = total_loss / len(val_loader)
    print(f"검증 데이터셋의 평균 손실(MSE): {avg_loss:.4f}")
    
    # 원래 스케일에서의 RMSE/MAE/R²
    rmse = mean_squared_error(y_true_all, y_pred_all)
    mae  = mean_absolute_error(y_true_all, y_pred_all)
    r2   = r2_score(y_true_all, y_pred_all)

    print(f"검증 MAE:  {mae:,.2f}")
    print(f"검증 RMSE: {rmse:,.2f}")
    print(f"검증 R²:   {r2:.4f}")

print("모델 평가 완료!")


모델 평가 시작...


 11%|█         | 1/9 [00:00<00:03,  2.13it/s]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


 22%|██▏       | 2/9 [00:00<00:03,  2.21it/s]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


...

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


 89%|████████▉ | 8/9 [00:03<00:00,  2.07it/s]

image_features : torch.Size([32, 32])
tab_features : torch.Size([32, 256])
combined_features : torch.Size([32, 288])
output : torch.Size([32, 1])


100%|██████████| 9/9 [00:04<00:00,  2.16it/s]

image_features : torch.Size([23, 32])
tab_features : torch.Size([23, 256])
combined_features : torch.Size([23, 288])
output : torch.Size([23, 1])
검증 데이터셋의 평균 손실(MSE): 153590.9245
검증 MAE:  154,421.50
검증 RMSE: 63,565,152,256.00
검증 R²:   -0.1253
모델 평가 완료!
